# PROJECT 1


In [1]:
import torch
import torchvision
torchvision.__version__

'0.20.1+cu124'

In [ ]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4393, done.
remote: Counting objects: 100% (1532/1532), done.
remote: Compressing objects: 100% (133/133), done.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
import os
import zipfile
from pathlib import Path
import requests

def download_data(source:str,
                  destination:str,
                  remove_source:bool=True)->Path:

                  data_path=Path("data/")
                  image_path=data_path/destination

                  if image_path.is_dir():
                    print(f"[INFO] {image_path} directory already exists, skipping download.")
                  else:
                    print(f"[INFO] {image_path} creating one.")
                    image_path.mkdir(parents=True,exist_ok=True)

                  target_file=Path(source).name
                  with open(data_path/target_file,"wb") as f:
                    request=requests.get(source)
                    print(f"[INFO] downloading {target_file} from {source}")
                    f.write(request.content)

                  with zipfile.ZipFile(data_path/target_file,"r") as zip_ref:
                    print(f"[INFO] unzipping {target_file}")
                    zip_ref.extractall(image_path)

                  if remove_source:
                    os.remove(data_path/target_file)

                  return image_path


image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

In [ ]:

train_dir=image_path/"train"
test_dir=image_path/"test"

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

from torchvision import transforms
manual_transforms=transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      normalize])
from going_modular.going_modular import data_setup
train_dataloaders,test_dataloaders,class_names=data_setup.create_dataloaders(train_dir=train_dir,test_dir=test_dir,transform=manual_transforms,batch_size=32)

In [ ]:
train_dataloaders,test_dataloaders,class_names

In [ ]:
import torchvision
weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT

automatic_transforms=weights.transforms()

automatic_transforms

train_dataloaders,test_dataloaders,class_names=data_setup.create_dataloaders(train_dir=train_dir,test_dir=test_dir,transform=automatic_transforms,batch_size=32)

In [ ]:
weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
model=torchvision.models.efficientnet_b0(weights=weights).to(device)
model

In [ ]:
for param in model.parameters():
  param.requires_grad=False

model.classifier=nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,out_features=len(class_names),bias=True)
).to(device)

In [ ]:
from torchinfo import summary
summary(model=model,
        input_size=(32,3,224,224),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        verbose=0,
        row_settings=["var_names"]
)

In [ ]:
try:
    from torch.utils.tensorboard import SummaryWriter
except:
    print("[INFO] Couldn't find tensorboard... installing it.")
    !pip install -q tensorboard
    from torch.utils.tensorboard import SummaryWriter


# Create a writer with all default settings
writer = SummaryWriter()

In [ ]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step

# Import train() function from:
# https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/going_modular/engine.py
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]}
      For example if training for epochs=2:
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        if isinstance(train_loss, torch.Tensor):
            train_loss = train_loss.mean().item()  # Ensure it's a scalar
        if isinstance(train_acc, torch.Tensor):
            train_acc = train_acc.mean().item()
        if isinstance(test_loss, torch.Tensor):
            test_loss = test_loss.mean().item()
        if isinstance(test_acc, torch.Tensor):
            test_acc = test_acc.mean().item()

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### New: Experiment tracking ###
        # Add loss results to SummaryWriter
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        # Add accuracy results to SummaryWriter
        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc},
                           global_step=epoch)

        # Track the PyTorch model architecture
        writer.add_graph(model=model,
                         # Pass in an example input
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))

    # Close the writer
    writer.close()

    ### End new ###

    # Return the filled results at the end of the epochs
    return results

In [ ]:
import torch.nn as nn
import torch.optim as optim

# ... (rest of your code) ...

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Example loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Example optimizer

# Then call the train function with the defined loss_fn and optimizer:
results = train(model=model,
                train_dataloader=train_dataloaders,
                test_dataloader=test_dataloaders,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

In [ ]:
results

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs